# Cloud Autoscaling using Reinforcement Learning

## Project: Comparing SARSA and Q-Learning for Cloud Resource Management

### Team 3
- Balasubramanyam, Srivatsa (mhe3sy)
- Healy, Ryan (rah5ff)
- McGregor, Bruce (bm3pk)

### University of Virginia
### Reinforcement Learning - Fall 2025

---

## Overview

This notebook implements and compares SARSA and Q-Learning algorithms for cloud resource autoscaling. We explore:

1. **Environment Design**: Gymnasium-compatible simulator with realistic workload patterns
2. **RL Agents**: Implementation of SARSA and Q-Learning with ε-greedy exploration
3. **Baseline Policies**: Simple threshold-based and reactive policies for comparison
4. **Experiments**: Systematic comparison of hyperparameters and algorithms
5. **Analysis**: Performance evaluation focusing on SLA violations vs. cost trade-offs

## 1. Setup and Imports

In [ ]:
!pip install gymnasium numpy pandas matplotlib seaborn -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gymnasium as gym
from typing import Tuple, Dict, Optional
import pickle
import os

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seeds
np.random.seed(42)

print("All imports successful!")

## 2. Cloud Autoscaling Environment

### State Space
- **Utilization Level**: 0 (low <40%), 1 (medium 40-80%), 2 (high >80%)
- **Capacity Level**: 0-4 (representing 1-5 capacity units)
- **Demand Trend**: 0 (falling), 1 (flat), 2 (rising)

### Action Space
- **0**: Scale down (remove capacity)
- **1**: Hold steady (no change)
- **2**: Scale up (add capacity)

### Reward Function
- **+10**: Optimal utilization (40-80%)
- **+5**: Efficiency bonus (60-70%)
- **-50+**: SLA violation penalty (utilization >90%)
- **-5**: Wasted capacity penalty (utilization <20%)
- **-2**: Capacity change penalty
- **-0.5×capacity**: Cost penalty per step

In [ ]:
# Load the environment code
exec(open('/home/cloud_autoscaling_env.py').read())

print("✓ Environment loaded successfully!")

### 2.1 Generate Workload Data

In [ ]:
# Generate synthetic workload with realistic patterns
def generate_workload(length=1000, seed=42):
    """Generate synthetic cloud workload with daily patterns and spikes."""
    np.random.seed(seed)
    t = np.linspace(0, 4 * np.pi, length)
    
    # Combine multiple patterns
    daily_pattern = 50 + 30 * np.sin(t)  # Daily cycle
    weekly_pattern = 10 * np.sin(t / 7)  # Weekly variation
    noise = np.random.normal(0, 5, length)  # Random noise
    spikes = np.random.choice([0, 20], size=length, p=[0.95, 0.05])  # Occasional spikes
    
    workload = daily_pattern + weekly_pattern + noise + spikes
    workload = np.clip(workload, 10, 100)
    
    return workload

# Generate workload
workload_data = generate_workload(length=1000)

# Visualize workload
plt.figure(figsize=(15, 5))
plt.plot(workload_data, alpha=0.7, linewidth=1)
plt.xlabel('Time Step')
plt.ylabel('Demand')
plt.title('Synthetic Cloud Workload Pattern')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Workload statistics:")
print(f"  Mean: {np.mean(workload_data):.2f}")
print(f"  Std: {np.std(workload_data):.2f}")
print(f"  Min: {np.min(workload_data):.2f}")
print(f"  Max: {np.max(workload_data):.2f}")

### 2.2 Test Environment

In [ ]:
# Create and test environment
env = CloudAutoscalingEnv(workload_data=workload_data, seed=42)

print("Environment Details:")
print(f"  State space: {env.observation_space}")
print(f"  Action space: {env.action_space}")
print(f"  Max capacity: {env.max_capacity}")
print(f"  Min capacity: {env.min_capacity}")

# Test episode
state, info = env.reset()
print(f"\nInitial state: {state}")
print(f"Initial info: {info}")

# Take a few steps
print("\nTaking 5 random steps:")
for i in range(5):
    action = env.action_space.sample()
    next_state, reward, terminated, truncated, info = env.step(action)
    action_name = ['Scale Down', 'Hold', 'Scale Up'][action]
    print(f"  Step {i+1}: Action={action_name}, Reward={reward:.2f}, "
          f"Utilization={info['utilization']:.2%}, Capacity={info['capacity']}")

print("\n✓ Environment working correctly!")

## 3. Baseline Policies

Before implementing RL algorithms, let's establish baseline performance using simple policies.

In [ ]:
# Load baseline policies
exec(open('/home/baseline_policies.py').read())

print("Baseline policies loaded!")

In [ ]:
# Compare all baseline policies
baseline_results = compare_baselines(env, n_episodes=100)

## 4. Q-Learning Implementation

### Q-Learning Update Rule
$$Q(s,a) \leftarrow Q(s,a) + \alpha[r + \gamma \max_{a'} Q(s',a') - Q(s,a)]$$

Q-Learning is an **off-policy** algorithm that learns the optimal policy while following an exploration policy.

In [ ]:
# Load Q-Learning agent
exec(open('/home/q_learning_agent.py').read())

print("Q-Learning agent loaded!")

In [ ]:
# Create Q-Learning agent
q_agent = QLearningAgent(
    state_space_shape=(3, 5, 3),
    n_actions=3,
    learning_rate=0.1,
    discount_factor=0.95,
    epsilon=1.0,
    epsilon_decay=0.995,
    epsilon_min=0.01,
    seed=42
)

print("Q-Learning Agent Configuration:")
print(f"  Learning rate (α): {q_agent.learning_rate}")
print(f"  Discount factor (γ): {q_agent.discount_factor}")
print(f"  Initial epsilon (ε): {q_agent.epsilon}")
print(f"  Epsilon decay: {q_agent.epsilon_decay}")
print(f"  Q-table shape: {q_agent.q_table.shape}")
print(f"  Total Q-values: {q_agent.q_table.size}")

In [ ]:
# Train Q-Learning agent
print("Training Q-Learning Agent...\n")
q_agent, q_metrics = train_q_learning(
    env, 
    q_agent, 
    n_episodes=1000, 
    verbose=True, 
    verbose_freq=100
)

print("\n✓ Q-Learning training complete!")

In [ ]:
# Evaluate Q-Learning agent
q_eval = eval_q(env, q_agent, n_episodes=100, verbose=True)

## 5. SARSA Implementation

### SARSA Update Rule
$$Q(s,a) \leftarrow Q(s,a) + \alpha[r + \gamma Q(s',a') - Q(s,a)]$$

SARSA is an **on-policy** algorithm that learns the value of the policy being followed (including exploration).

In [ ]:
# Load SARSA agent
exec(open('/home/sarsa_agent.py').read())

print("SARSA agent loaded!")

In [ ]:
# Create SARSA agent
sarsa_agent = SARSAAgent(
    state_space_shape=(3, 5, 3),
    n_actions=3,
    learning_rate=0.1,
    discount_factor=0.95,
    epsilon=1.0,
    epsilon_decay=0.995,
    epsilon_min=0.01,
    seed=42
)

print("SARSA Agent Configuration:")
print(f"  Learning rate (α): {sarsa_agent.learning_rate}")
print(f"  Discount factor (γ): {sarsa_agent.discount_factor}")
print(f"  Initial epsilon (ε): {sarsa_agent.epsilon}")
print(f"  Epsilon decay: {sarsa_agent.epsilon_decay}")
print(f"  Q-table shape: {sarsa_agent.q_table.shape}")

In [ ]:
# Train SARSA agent
print("Training SARSA Agent...\n")
sarsa_agent, sarsa_metrics = train_sarsa(
    env, 
    sarsa_agent, 
    n_episodes=1000, 
    verbose=True, 
    verbose_freq=100
)

print("SARSA training complete!")

In [ ]:
# Evaluate SARSA agent
sarsa_eval = eval_sarsa(env, sarsa_agent, n_episodes=100, verbose=True)

## 6. Results Analysis and Visualization

### 6.1 Training Curves

In [ ]:
# Plot training curves
def moving_average(data, window=50):
    return np.convolve(data, np.ones(window)/window, mode='valid')

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Rewards
ax = axes[0, 0]
q_rewards = q_metrics['episode_rewards']
sarsa_rewards = sarsa_metrics['episode_rewards']
ax.plot(moving_average(q_rewards), label='Q-Learning', alpha=0.8, linewidth=2)
ax.plot(moving_average(sarsa_rewards), label='SARSA', alpha=0.8, linewidth=2)
ax.set_xlabel('Episode', fontsize=12)
ax.set_ylabel('Reward (Moving Avg)', fontsize=12)
ax.set_title('Training Rewards', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Plot 2: SLA Violations
ax = axes[0, 1]
q_sla = q_metrics['sla_violations']
sarsa_sla = sarsa_metrics['sla_violations']
ax.plot(moving_average(q_sla), label='Q-Learning', alpha=0.8, linewidth=2)
ax.plot(moving_average(sarsa_sla), label='SARSA', alpha=0.8, linewidth=2)
ax.set_xlabel('Episode', fontsize=12)
ax.set_ylabel('SLA Violations (Moving Avg)', fontsize=12)
ax.set_title('SLA Violations During Training', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Plot 3: Costs
ax = axes[1, 0]
q_costs = q_metrics['costs']
sarsa_costs = sarsa_metrics['costs']
ax.plot(moving_average(q_costs), label='Q-Learning', alpha=0.8, linewidth=2)
ax.plot(moving_average(sarsa_costs), label='SARSA', alpha=0.8, linewidth=2)
ax.set_xlabel('Episode', fontsize=12)
ax.set_ylabel('Cost (Moving Avg)', fontsize=12)
ax.set_title('Total Cost During Training', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Plot 4: Episode Lengths
ax = axes[1, 1]
q_lengths = q_metrics['episode_lengths']
sarsa_lengths = sarsa_metrics['episode_lengths']
ax.plot(moving_average(q_lengths), label='Q-Learning', alpha=0.8, linewidth=2)
ax.plot(moving_average(sarsa_lengths), label='SARSA', alpha=0.8, linewidth=2)
ax.set_xlabel('Episode', fontsize=12)
ax.set_ylabel('Episode Length (Moving Avg)', fontsize=12)
ax.set_title('Episode Lengths', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 6.2 Performance Comparison

In [ ]:
# Create comparison table
comparison_data = {
    'Method': [
        'Random',
        'Threshold',
        'Reactive',
        'Proactive',
        'Conservative',
        'Q-Learning',
        'SARSA'
    ],
    'Mean Reward': [
        baseline_results['RandomPolicy']['mean_reward'],
        baseline_results['ThresholdPolicy']['mean_reward'],
        baseline_results['ReactivePolicy']['mean_reward'],
        baseline_results['ProactivePolicy']['mean_reward'],
        baseline_results['ConservativePolicy']['mean_reward'],
        q_eval['mean_reward'],
        sarsa_eval['mean_reward']
    ],
    'SLA Violations': [
        baseline_results['RandomPolicy']['mean_sla_violations'],
        baseline_results['ThresholdPolicy']['mean_sla_violations'],
        baseline_results['ReactivePolicy']['mean_sla_violations'],
        baseline_results['ProactivePolicy']['mean_sla_violations'],
        baseline_results['ConservativePolicy']['mean_sla_violations'],
        q_eval['mean_sla_violations'],
        sarsa_eval['mean_sla_violations']
    ],
    'Mean Cost': [
        baseline_results['RandomPolicy']['mean_cost'],
        baseline_results['ThresholdPolicy']['mean_cost'],
        baseline_results['ReactivePolicy']['mean_cost'],
        baseline_results['ProactivePolicy']['mean_cost'],
        baseline_results['ConservativePolicy']['mean_cost'],
        q_eval['mean_cost'],
        sarsa_eval['mean_cost']
    ],
    'Utilization': [
        baseline_results['RandomPolicy']['mean_utilization'],
        baseline_results['ThresholdPolicy']['mean_utilization'],
        baseline_results['ReactivePolicy']['mean_utilization'],
        baseline_results['ProactivePolicy']['mean_utilization'],
        baseline_results['ConservativePolicy']['mean_utilization'],
        q_eval['mean_utilization'],
        sarsa_eval['mean_utilization']
    ]
}

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.round(2)


print(df_comparison.to_string(index=False))

In [ ]:
# Bar chart comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

methods = df_comparison['Method'].tolist()
colors = ['skyblue']*5 + ['coral', 'coral']

# Plot 1: Rewards
axes[0].bar(range(len(methods)), df_comparison['Mean Reward'], color=colors, alpha=0.8, edgecolor='black')
axes[0].set_xticks(range(len(methods)))
axes[0].set_xticklabels(methods, rotation=45, ha='right')
axes[0].set_ylabel('Mean Reward', fontsize=12)
axes[0].set_title('Average Reward Comparison', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Plot 2: SLA Violations
axes[1].bar(range(len(methods)), df_comparison['SLA Violations'], color=colors, alpha=0.8, edgecolor='black')
axes[1].set_xticks(range(len(methods)))
axes[1].set_xticklabels(methods, rotation=45, ha='right')
axes[1].set_ylabel('Mean SLA Violations', fontsize=12)
axes[1].set_title('SLA Violations Comparison', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Plot 3: Costs
axes[2].bar(range(len(methods)), df_comparison['Mean Cost'], color=colors, alpha=0.8, edgecolor='black')
axes[2].set_xticks(range(len(methods)))
axes[2].set_xticklabels(methods, rotation=45, ha='right')
axes[2].set_ylabel('Mean Cost', fontsize=12)
axes[2].set_title('Cost Comparison', fontsize=14, fontweight='bold')
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 6.3 Policy Visualization

In [ ]:
# Visualize learned policies
def visualize_policy(agent, agent_name):
    """Visualize the learned policy as a heatmap."""
    policy = agent.get_policy()
    action_names = ['Scale Down', 'Hold', 'Scale Up']
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    trend_names = ['Falling Demand', 'Flat Demand', 'Rising Demand']
    
    for trend_idx, ax in enumerate(axes):
        # Extract policy for this trend
        policy_slice = policy[:, :, trend_idx]
        
        # Create heatmap
        im = ax.imshow(policy_slice, cmap='RdYlGn', aspect='auto', vmin=0, vmax=2)
        
        # Set ticks and labels
        ax.set_xticks(range(5))
        ax.set_yticks(range(3))
        ax.set_xticklabels(['1', '2', '3', '4', '5'])
        ax.set_yticklabels(['Low', 'Medium', 'High'])
        ax.set_xlabel('Capacity Level', fontsize=12)
        ax.set_ylabel('Utilization Level', fontsize=12)
        ax.set_title(trend_names[trend_idx], fontsize=13, fontweight='bold')
        
        # Add text annotations
        for i in range(3):
            for j in range(5):
                action = policy_slice[i, j]
                ax.text(j, i, action_names[action], ha='center', va='center',
                       fontsize=9, fontweight='bold', color='white' if action == 1 else 'black')
    
    plt.suptitle(f'{agent_name} Policy', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

# Visualize both policies
visualize_policy(q_agent, 'Q-Learning')
visualize_policy(sarsa_agent, 'SARSA')

## 7. Hyperparameter Experiments

### 7.1 Experiment: Different Exploration Rates

In [ ]:
# Compare different epsilon decay rates
epsilon_decays = [0.999, 0.995, 0.99, 0.98]
exploration_results = {}

print("Testing different exploration rates...\n")

for decay in epsilon_decays:
    print(f"Training with epsilon_decay={decay}...")
    
    # Train Q-Learning
    agent = QLearningAgent(
        state_space_shape=(3, 5, 3),
        n_actions=3,
        learning_rate=0.1,
        discount_factor=0.95,
        epsilon=1.0,
        epsilon_decay=decay,
        epsilon_min=0.01,
        seed=42
    )
    
    agent, metrics = train_q_learning(env, agent, n_episodes=500, verbose=False)
    eval_metrics = eval_q(env, agent, n_episodes=50, verbose=False)
    
    exploration_results[decay] = {
        'training': metrics,
        'evaluation': eval_metrics
    }
    
    print(f"  Final Reward: {eval_metrics['mean_reward']:.2f}")
    print(f"  SLA Violations: {eval_metrics['mean_sla_violations']:.2f}\n")


In [ ]:
# Plot exploration results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

for decay, results in exploration_results.items():
    rewards = results['training']['episode_rewards']
    axes[0].plot(moving_average(rewards, 20), label=f'decay={decay}', alpha=0.7)

axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Reward (Moving Avg)')
axes[0].set_title('Training Rewards vs Epsilon Decay')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Bar chart of final performance
decays = list(exploration_results.keys())
final_rewards = [exploration_results[d]['evaluation']['mean_reward'] for d in decays]
final_sla = [exploration_results[d]['evaluation']['mean_sla_violations'] for d in decays]

x = np.arange(len(decays))
width = 0.35

axes[1].bar(x - width/2, final_rewards, width, label='Reward', alpha=0.8)
axes[1].bar(x + width/2, final_sla, width, label='SLA Violations', alpha=0.8)
axes[1].set_xlabel('Epsilon Decay')
axes[1].set_ylabel('Value')
axes[1].set_title('Final Performance by Epsilon Decay')
axes[1].set_xticks(x)
axes[1].set_xticklabels(decays)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 7.2 Experiment: Different Learning Rates

In [ ]:
# Compare different learning rates
learning_rates = [0.01, 0.05, 0.1, 0.3]
lr_results = {}

print("Testing different learning rates...\n")

for lr in learning_rates:
    print(f"Training with learning_rate={lr}...")
    
    # Train Q-Learning
    agent = QLearningAgent(
        state_space_shape=(3, 5, 3),
        n_actions=3,
        learning_rate=lr,
        discount_factor=0.95,
        epsilon=1.0,
        epsilon_decay=0.995,
        epsilon_min=0.01,
        seed=42
    )
    
    agent, metrics = train_q_learning(env, agent, n_episodes=500, verbose=False)
    eval_metrics = eval_q(env, agent, n_episodes=50, verbose=False)
    
    lr_results[lr] = {
        'training': metrics,
        'evaluation': eval_metrics
    }
    
    print(f"  Final Reward: {eval_metrics['mean_reward']:.2f}")
    print(f"  SLA Violations: {eval_metrics['mean_sla_violations']:.2f}\n")


In [ ]:
# Plot learning rate results
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

for lr, results in lr_results.items():
    rewards = results['training']['episode_rewards']
    ax.plot(moving_average(rewards, 20), label=f'α={lr}', alpha=0.7, linewidth=2)

ax.set_xlabel('Episode', fontsize=12)
ax.set_ylabel('Reward (Moving Avg)', fontsize=12)
ax.set_title('Training Rewards vs Learning Rate', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Summary table
lr_summary = pd.DataFrame([
    {
        'Learning Rate': lr,
        'Mean Reward': lr_results[lr]['evaluation']['mean_reward'],
        'SLA Violations': lr_results[lr]['evaluation']['mean_sla_violations'],
        'Mean Cost': lr_results[lr]['evaluation']['mean_cost']
    }
    for lr in learning_rates
])

print(lr_summary.to_string(index=False))

## 8. Discussion and Key Findings

### Q-Learning vs SARSA

**Key Differences:**
1. **On-policy vs Off-policy**: SARSA learns the policy it's following (on-policy), while Q-Learning learns the optimal policy regardless of the exploration policy (off-policy)
2. **Update Rule**: SARSA uses the actual next action taken, Q-Learning uses the maximum Q-value of the next state
3. **Exploration Impact**: SARSA is more cautious as it considers its exploration strategy in learning

### Performance Insights

1. **Reward Optimization**: Both RL methods significantly outperform baseline policies
2. **SLA Compliance**: RL agents learn to anticipate demand and scale proactively
3. **Cost Efficiency**: Learned policies balance performance and cost better than threshold rules
4. **Trend Feature**: Including demand trend in state space improves proactive scaling

### Hyperparameter Sensitivity

1. **Exploration Rate**: Slower epsilon decay (0.999) allows better exploration but slower convergence
2. **Learning Rate**: Moderate learning rates (0.1-0.3) work best; too low is slow, too high is unstable
3. **Discount Factor**: Higher gamma (0.95-0.99) helps with long-term planning

### Practical Implications

1. **Real-world Deployment**: RL agents can adapt to changing workload patterns
2. **Safety Considerations**: On-policy SARSA may be safer for production due to conservative learning
3. **Computational Cost**: Q-Learning converges faster, making it suitable for online learning

## 9. Save Results and Models

In [ ]:
# Create directories
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)

# Save trained agents
q_agent.save('models/q_learning_agent.pkl')
sarsa_agent.save('models/sarsa_agent.pkl')

# Save all results
all_results = {
    'baseline_results': baseline_results,
    'q_learning': {
        'training': q_metrics,
        'evaluation': q_eval
    },
    'sarsa': {
        'training': sarsa_metrics,
        'evaluation': sarsa_eval
    },
    'exploration_experiment': exploration_results,
    'learning_rate_experiment': lr_results
}

with open('results/all_results.pkl', 'wb') as f:
    pickle.dump(all_results, f)

print("✓ Models and results saved successfully!")
print("  - Models saved in 'models/' directory")
print("  - Results saved in 'results/' directory")

## 10. Conclusion

This project successfully demonstrated that reinforcement learning algorithms (SARSA and Q-Learning) can make smarter cloud autoscaling decisions than simple threshold-based policies. Key achievements:

1. ✓ Built a realistic cloud autoscaling simulator with Gymnasium interface
2. ✓ Implemented both SARSA and Q-Learning with ε-greedy exploration
3. ✓ Incorporated demand trend feature for proactive scaling
4. ✓ Systematically compared hyperparameters and exploration strategies
5. ✓ Demonstrated superior performance vs. baseline policies
6. ✓ Achieved optimal SLA compliance while minimizing costs

### Future Work

1. Test with real-world cloud traces (Google, Alibaba datasets)
2. Implement more advanced RL algorithms (DQN, PPO)
3. Add multi-resource scaling (CPU, memory, network)
4. Incorporate safety constraints and risk-aware learning
5. Deploy and validate in production environment